In [ ]:
using Pkg
Pkg.add(url="https://github.com/toubinaattori/thesis")
using DecisionProgramming
using JuMP
using GLPK

diagram = InfluenceDiagram()
diagram.Cost = []

N = 3
M = 1
c_k = rand(N)
fortification(k, a) = [c_k[k], 0][a]
b = 0.03

add_node!(diagram, ChanceNode("L", [], ["high", "low"]))

for i in 1:N
    add_node!(diagram, ChanceNode("R$i", ["L"], ["high", "low"]))
end

for i in 1:M
    add_node!(diagram, DecisionNode("A$i",map(x -> "R$x", 1:N), ["yes", "no"],map(x -> "R$x", 1:N)))
end

add_node!(diagram, ChanceNode("F", ["L", ["A$i" for i in 1:M]...], ["failure", "success"]))

add_node!(diagram, ValueNode("T", ["F", ["A$i" for i in 1:M]...]))

for i in 1:N
    for j in 1:M
        add_costs!(diagram,Costs(("R$i","A$j"), 1))
    end
end

generate_arcs!(diagram)

X_L = [rand(), 0]
X_L[2] = 1.0 - X_L[1]
add_probabilities!(diagram, "L", X_L)

for i in 1:N
    x_R, y_R = rand(2)
    X_R = ProbabilityMatrix(diagram, "R$i")
    X_R["high", "high"] = max(x_R, 1-x_R)
    X_R["high", "low"] = 1 - max(x_R, 1-x_R)
    X_R["low", "low"] = max(y_R, 1-y_R)
    X_R["low", "high"] = 1-max(y_R, 1-y_R)
    add_probabilities!(diagram, "R$i", X_R)
end


X_F = ProbabilityMatrix(diagram, "F")
x_F, y_F = rand(2)
for s in paths([State(2) for i in 1:M])
    denominator = exp(b * sum(fortification(k, a) for (k, a) in enumerate(s)))
    s1 = [s...]
    X_F[1, s1..., 1] = max(x_F, 1-x_F) / denominator
    X_F[1, s..., 2] = 1.0 - X_F[1, s..., 1]
    X_F[2, s..., 1] = min(y_F, 1-y_F) / denominator
    X_F[2, s..., 2] = 1.0 - X_F[2, s..., 1]
end
add_probabilities!(diagram, "F", X_F)



Y_T = UtilityMatrix(diagram, "T")
for s in paths([State(2) for i in 1:M])
    cost = sum(-fortification(k, a) for (k, a) in enumerate(s))
    Y_T[1, s...] = 0 + cost
    Y_T[2, s...] = 100 + cost
end
add_utilities!(diagram, "T", Y_T)

generate_diagram!(diagram)

model2 = Model()
z = DecisionVariables(model2, diagram)
x_s = PathCompatibilityVariables(model2, diagram, z, names=true, name = "s", probability_cut = false)
x_x = ActiveDecisionPathVariables(model2,diagram,z,x_s,names=true,name="x")
EV = expected_value(model2, diagram, x_s,x_x)
@objective(model2, Max, EV)

set_optimizer(model2, GLPK.Optimizer)
optimize!(model2)

for (d, z_d) in zip(z.D, z.z)
    println("-------------")
    println("Decision strategy at node")
    println(d)
    println("-------------")
    for s in z_d
        println(s)
        println(value.(s))
    end
end